# Using MoviePy for Good (and Evil\*) 

Lorem Ipsum is simply dummy text of the printing and typesetting industry. Lorem Ipsum has been the industry's standard dummy text ever since the 1500s, when an unknown printer took a galley of type and scrambled it to make a type specimen book. It has survived not only five centuries, but also the leap into electronic typesetting, remaining essentially unchanged. It was popularised in the 1960s with the release of Letraset sheets containing Lorem Ipsum passages, and more recently with desktop publishing software like Aldus PageMaker including versions of Lorem Ipsum.


## Data Visualizations

You can use MoviePy for data visualizations. Here's an example by [Zulko](http://zulko.github.io/blog/2014/11/29/data-animations-with-python-and-moviepy/).

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from moviepy.video.io.bindings import mplfig_to_npimage
import moviepy.editor as mpy
from IPython.display import Image

# DRAW A FIGURE WITH MATPLOTLIB

duration = 2

fig_mpl, ax = plt.subplots(1,figsize=(5,3), facecolor='white')
xx = np.linspace(-2,2,200) # the x vector
zz = lambda d: np.sinc(xx**2)+np.sin(xx+d) # the (changing) z vector
ax.set_title("Elevation in y=0")
ax.set_ylim(-1.5,2.5)
line, = ax.plot(xx, zz(0), lw=3)

# ANIMATE WITH MOVIEPY (UPDATE THE CURVE FOR EACH t). MAKE A GIF.

def make_frame_mpl(t):
    line.set_ydata( zz(2*np.pi*t/duration))  # <= Update the curve
    return mplfig_to_npimage(fig_mpl) # RGB image of the figure

animation =mpy.VideoClip(make_frame_mpl, duration=duration)
animation.write_gif("./sinc_mpl.gif", fps=20)

Image(url="sinc_mpl.gif") 

## Using PyGame

Sometimes you need somthing more advanced than rendering a mask or text on a video. As long as you can generate an 2D NumPy array of colors, you can anything!

Here's an example using PyGame:

In [ ]:
import os
import pygame
import moviepy.editor as mpy
from IPython.display import HTML

# Headless SDL
os.environ['SDL_VIDEODRIVER'] = 'dummy'

# Start up pygame
pygame.init()

# Going to hide the main display (make it small)
pygame.display.set_mode((1,1))

# Buffer we're going to draw changes in
screen = pygame.Surface((1024, 768), pygame.SRCALPHA, 32)

# Get your own duck
duckface = pygame.image.load("duck.bmp")
position = duckface.get_rect()

# Function to get a frame 
# Returns a NumPy-style array size WIDTHxHEIGHTx3
# Represeting a 2D RGB field. (Each component is 8-bit)

def next_frame(dt):
    global position # globals, lol

    # Translate by (2, 0)
    position = position.move(2, 0)

    # Fill the screen with black
    screen.fill(0x00000000)

    # Draw duckface
    screen.blit(duckface, position)

    # Bring the image from the other buffer to 
    # main
    pygame.display.flip()

    # For some reason, we have to do this or
    # else the image comes out flipped
    newscreen = pygame.transform.rotate(screen, 90)

    # Reverse the columns of the pixel array
    # We could do it with MoviePy with .fx(vfx.mirror_y)
    return pygame.surfarray.array3d(newscreen)[:,::-1]
    

duration = 1 # in seconds
animation = mpy.VideoClip(next_frame, duration=duration)
animation.write_videofile("./video.mp4", fps=60) # 60fps or die

HTML("""
<video width="1024" height="768" controls>
  <source src="./video.mp4" type="video/mp4">
</video>
""")


## Dependencies

I've only tested this on macOS. Please update the instructions if you find yourself installing this on other platforms. Install with `pip install -r requirements.txt`

```
# requirements.txt

appnope==0.1.0
apptools==4.4.0
configobj==5.0.6
cycler==0.10.0
decorator==4.0.10
entrypoints==0.2.2
imageio==1.5
ipykernel==4.4.1
ipython==5.1.0
ipython-genutils==0.1.0
ipywidgets==5.2.2
Jinja2==2.8
jsonschema==2.5.1
jupyter==1.0.0
jupyter-client==4.3.0
jupyter-console==5.0.0
jupyter-core==4.1.1
MarkupSafe==0.23
matplotlib==1.5.2
mistune==0.7.3
moviepy==0.2.2.11
nbconvert==4.2.0
nbformat==4.1.0
notebook==4.2.2
numpy==1.11.1
pexpect==4.2.1
pickleshare==0.7.4
prompt-toolkit==1.0.7
ptyprocess==0.5.1
pyface==5.1.0
pygame==1.9.2b6
Pygments==2.1.3
pyparsing==2.1.8
python-dateutil==2.5.3
pytz==2016.6.1
pyzmq==15.4.0
qtconsole==4.2.1
simplegeneric==0.8.1
six==1.10.0
terminado==0.6
tornado==4.4.1
tqdm==4.8.4
traitlets==4.2.2
traits==4.5.0
traitsui==5.1.0
wcwidth==0.1.7
widgetsnbextension==1.2.6
```